In [ ]:
import sys
import os

if 'google.colab' in sys.modules:
    if not os.path.exists("mlb-betting-prediction-pipeline"):
        !git clone https://github.com/gerardrobertkirwin/mlb-betting-prediction-pipeline.git
    os.chdir("mlb-betting-prediction-pipeline")
    !pip install -r requirements.txt -q

if "." not in sys.path:
    sys.path.append(".")

import pandas as pd
import numpy as np
import requests
import time
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

In [ ]:
from google.colab import drive
import os


drive.mount('/content/drive')

project_dir = "/content/drive/MyDrive/mlb-project-data"
os.makedirs(project_dir, exist_ok=True)

print(f"Google Drive Mounted at {project_dir}")

In [ ]:
class MLBStatsAPI:
  base_url = "https://statsapi.mlb.com/api/v1"

  def __init__(self):
    self.session = requests.Session()
    retries = Retry(total=3, backoff_factor=1, status_forcelist=[500,502,503,504])
    self.session.mount('https://', HTTPAdapter(max_retries=retries))

  def get_season_schedule(self, season: int) -> pd.DataFrame:
        """
        Fetch schedule. Try to get stats (Hits/Errors), but don't drop the game if they are missing.
        """
        url = f"{self.base_url}/schedule"

            'sportId': 1,
            'season': season,
            'gameType': 'R',
            'hydrate': 'linescore'
        }

        print(f"Fetching schedule + stats for {season}...")
        try:
            response = self.session.get(url, params=params)
            response.raise_for_status()
            data = response.json()
        except Exception as e:
            print(f"API Error: {e}")
            return pd.DataFrame()

        games_list = []

        if 'dates' not in data:
            return pd.DataFrame()

        for date_obj in data['dates']:
            date = date_obj['date']
            for game in date_obj['games']:

                home_team_data = game['teams']['home']
                away_team_data = game['teams']['away']


                if 'score' not in home_team_data or 'score' not in away_team_data:
                    continue


                linescore = game.get('linescore', {})

                ls_home = linescore.get('teams', {}).get('home', {})
                ls_away = linescore.get('teams', {}).get('away', {})

                home_hits = ls_home.get('hits', 0)
                home_errors = ls_home.get('errors', 0)
                away_hits = ls_away.get('hits', 0)
                away_errors = ls_away.get('errors', 0)

                games_list.append({
                    'game_id': game['gamePk'],
                    'date': date,
                    'home_team': home_team_data['team']['name'],
                    'away_team': away_team_data['team']['name'],
                    'home_score': home_team_data['score'], # Guaranteed
                    'away_score': away_team_data['score'], # Guaranteed
                    'home_hits': home_hits,   # Might be 0 if API gets lazy
                    'home_errors': home_errors,
                    'away_hits': away_hits,
                    'away_errors': away_errors
                })

        return pd.DataFrame(games_list)

  def get_game_boxscore(self, game_id) -> dict:
      """Fetch detailed boxscore for a game."""
      url = f"{self.base_url}/game/{game_id}/boxscore"

      response = self.session.get(url)
      response.raise_for_status()
      data = response.json()

      teams = data.get('teams', {})
      home = teams.get('home', {}).get('teamStats', {}).get('batting', {})
      away = teams.get('away', {}).get('teamStats', {}).get('batting', {})

      return {
          'game_id': game_id,
          'home_hits': home.get('hits'),
          'home_errors': teams.get('home', {}).get('teamStats', {}).get('fielding', {}).get('errors'),
          'away_hits': away.get('hits'),
          'away_errors': teams.get('away', {}).get('teamStats', {}).get('fielding', {}).get('errors')
              }


In [ ]:
mlb = MLBStatsAPI()

df_schedule = mlb.get_season_schedule(2023)

print("Fetching boxscores for first 5 games...")
stats_list = []
for game_id in df_schedule['game_id'].head(5):
    stats = mlb.get_game_boxscore(game_id)
    stats_list.append(stats)
    time.sleep(0.2)

df_stats = pd.DataFrame(stats_list)
print(df_stats)

In [ ]:
df_mlb = mlb.get_season_schedule(2023)

In [ ]:
df_mlb.head()

In [ ]:
import json
import os

class BettingDataLoader:
  def __init__(self, filepath:str):
    self.filepath = filepath

  def load_odds(self, target_book: str = 'bet365') -> pd.DataFrame:
    print(f"Loading odds from {self.filepath} using {target_book}...")

    if not os.path.exists(self.filepath):
      raise FileNotFoundError(f"Could not find file: {self.filepath}")

    with open(self.filepath, 'r') as f:
      raw_data = json.load(f)

    if isinstance(raw_data, list):
      return pd.json_normalize(raw_data, sep='_')

    elif isinstance(raw_data, dict):
      games_list = []

      for date, games in raw_data.items():
          for game in games:
              view = game.get('gameView', {})

              g_type = view.get('gameType', 'R')
              if g_type != 'R':
                  continue


              odds_section = game.get('odds', {})
              moneylines = odds_section.get('moneyline', [])

              selected_book = None

              for book in moneylines:
                  if book.get('sportsbook', '').lower() == target_book.lower():
                      selected_book = book
                      break

              if selected_book is None and moneylines:
                  fallbacks = ['pinnacle', 'caesars', 'draftkings', 'fanduel']
                  for fb in fallbacks:
                      for book in moneylines:
                          # FIX: Added .lower() here too
                          if book.get('sportsbook', '').lower() == fb:
                              selected_book = book
                              break
                      if selected_book: break

                  if selected_book is None:
                      selected_book = moneylines[0]

              game_info = {
                  'date': date,
                  'away_team_abbr': view.get('awayTeam', {}).get('shortName'),
                  'away_score': view.get('awayTeamScore'),
                  'home_team_abbr': view.get('homeTeam', {}).get('shortName'),
                  'home_score': view.get('homeTeamScore'),
                  'game_type': g_type # Saved for audit
              }

              if selected_book:
                  current = selected_book.get('currentLine', {})
                  game_info['home_moneyline'] = current.get('homeOdds')
                  game_info['away_moneyline'] = current.get('awayOdds')
                  game_info['sportsbook'] = selected_book.get('sportsbook')
              else:
                  game_info['home_moneyline'] = np.nan
                  game_info['away_moneyline'] = np.nan
                  game_info['sportsbook'] = None

              games_list.append(game_info)

      return pd.json_normalize(games_list, sep='_')

In [ ]:
loader = BettingDataLoader("/content/drive/MyDrive/mlb-project-data/mlb_odds_dataset.json")
df_odds = loader.load_odds()
df_odds.info()
print(f"Rows: {len(df_odds)}")
print(df_odds['sportsbook'].value_counts())


In [ ]:
df_odds.head()
df_odds[df_odds['date'] == '2023-03-30']

In [ ]:
df_mlb[df_mlb['date'] == '2023-03-30']

In [ ]:
def get_team_abbr(name: str) -> str:

    mapping = {
        'New York Yankees': 'NYY', 'NY Yankees': 'NYY', 'New York (AL)': 'NYY',
        'Boston Red Sox': 'BOS', 'Boston': 'BOS',
        'Tampa Bay Rays': 'TB', 'Tampa Bay': 'TB', 'Tampa': 'TB',
        'Toronto Blue Jays': 'TOR', 'Toronto': 'TOR',
        'Baltimore Orioles': 'BAL', 'Baltimore': 'BAL',

        'Cleveland Guardians': 'CLE', 'Cleveland Indians': 'CLE', 'Cleveland': 'CLE',
        'Chicago White Sox': 'CHW', 'Chi White Sox': 'CHW',
        'Detroit Tigers': 'DET', 'Detroit': 'DET',
        'Kansas City Royals': 'KC', 'Kansas City': 'KC',
        'Minnesota Twins': 'MIN', 'Minnesota': 'MIN',

        'Houston Astros': 'HOU', 'Houston': 'HOU',
        'Seattle Mariners': 'SEA', 'Seattle': 'SEA',
        'Texas Rangers': 'TEX', 'Texas': 'TEX',
        'Oakland Athletics': 'OAK', 'Oakland': 'OAK',
        'Los Angeles Angels': 'LAA', 'LA Angels': 'LAA', 'Anaheim': 'LAA',

        'Atlanta Braves': 'ATL', 'Atlanta': 'ATL',
        'New York Mets': 'NYM', 'NY Mets': 'NYM', 'New York (NL)': 'NYM',
        'Philadelphia Phillies': 'PHI', 'Philadelphia': 'PHI',
        'Miami Marlins': 'MIA', 'Miami': 'MIA', 'Florida Marlins': 'MIA',
        'Washington Nationals': 'WAS', 'Washington': 'WAS',

        'Chicago Cubs': 'CHC', 'Chi Cubs': 'CHC',
        'St. Louis Cardinals': 'STL', 'St. Louis': 'STL', 'St Louis': 'STL',
        'Milwaukee Brewers': 'MIL', 'Milwaukee': 'MIL',
        'Cincinnati Reds': 'CIN', 'Cincinnati': 'CIN',
        'Pittsburgh Pirates': 'PIT', 'Pittsburgh': 'PIT',

        'Los Angeles Dodgers': 'LAD', 'LA Dodgers': 'LAD',
        'San Diego Padres': 'SD', 'San Diego': 'SD',
        'San Francisco Giants': 'SF', 'San Francisco': 'SF',
        'Arizona Diamondbacks': 'ARI', 'Arizona': 'ARI',
        'Colorado Rockies': 'COL', 'Colorado': 'COL'
    }

    clean_name = name.strip()
    return mapping.get(clean_name, "UNKNOWN") # Return UNKNOWN if not found so we can debug

In [ ]:
df_mlb['date'] = pd.to_datetime(df_mlb['date'])
df_odds['date'] = pd.to_datetime(df_odds['date'])

df_mlb['home_abbr'] = df_mlb['home_team'].apply(get_team_abbr)
df_mlb['away_abbr'] = df_mlb['away_team'].apply(get_team_abbr)

df_odds['home_abbr'] = df_odds['home_team_abbr']
df_odds['away_abbr'] = df_odds['away_team_abbr']

In [ ]:
#Losing NYM games?

nym_api = df_mlb[ (df_mlb['home_team'] == 'NYM') | (df_mlb['away_team'] == 'NYM') ].copy()
nym_api['date_str'] = nym_api['date'].astype(str)

nym_odds = df_odds[ (df_odds['home_team_abbr'] == 'NYM') | (df_odds['away_team_abbr'] == 'NYM') ].copy()
nym_odds['date_str'] = nym_odds['date'].astype(str)

api_dates = set(nym_api['date_str'])
odds_dates = set(nym_odds['date_str'])

missing_dates = api_dates - odds_dates

print(f"Dates present in API but NOT in Odds for NYM: {len(missing_dates)}")
print(sorted(list(missing_dates)))

print("\n--- SAMPLE MISSING DATE CHECK ---")
if missing_dates:
    sample_date = list(missing_dates)[0]
    print(f"Checking Date: {sample_date}")
    print("API Data:")
    print(nym_api[nym_api['date_str'] == sample_date][['home_team', 'away_team', 'home_score', 'away_score']])
    print("\nOdds Data:")
    print(nym_odds[nym_odds['date_str'] == sample_date][['home_team_abbr', 'away_team_abbr', 'home_score', 'away_score']])

In [ ]:
#d.duplicated().value_counts()

In [ ]:
df_mlb = df_mlb.sort_values('date')

rows_before = len(df_mlb)
df_mlb = df_mlb.drop_duplicates(subset=['game_id'], keep='last')
rows_after = len(df_mlb)

print(f"Dropped {rows_before - rows_after} duplicate games.")

mask_valid = (df_mlb['home_score'] > 0) | (df_mlb['away_score'] > 0) | (df_mlb['home_hits'] > 0)
df_mlb = df_mlb[mask_valid].copy()

print(f"Cleaned MLB Data: {len(df_mlb)} rows.")

In [ ]:
id_counts = df_mlb['game_id'].value_counts()

duplicate_ids = id_counts[id_counts > 1]

print(f"Found {len(duplicate_ids)} duplicate Game IDs.")
print(duplicate_ids)

if len(duplicate_ids) > 0:
    sample_ids = duplicate_ids.head(7).index.tolist()

    bad_rows = df_mlb[df_mlb['game_id'].isin(sample_ids)].sort_values('game_id')
    print("\n--- INSPECTING DUPLICATES ---")
    print(bad_rows[['date', 'game_id', 'home_team', 'away_team', 'home_score', 'away_score']])

In [ ]:
df_merged = pd.merge(
    df_mlb,
    df_odds,
    how='left',
    left_on=['date', 'home_abbr', 'away_abbr'],
    right_on=['date', 'home_team_abbr', 'away_team_abbr'],
    suffixes=('', '_odds')
)


condition = (
    (df_merged['home_score'] == df_merged['home_score_odds']) &
    (df_merged['away_score'] == df_merged['away_score_odds'])
) | (df_merged['home_score_odds'].isna()) # Keep it even if we didn't find odds

df_final_clean = df_merged[condition].copy()


print(f"Input Games: {len(df_mlb)}")
print(f"Merged Games: {len(df_final_clean)}")

In [ ]:
df_2023_odds = df_odds[(df_odds['date'] >= '2023-03-30') & (df_odds['date'] <= '2023-10-02')]
df_2023_odds

In [ ]:
debug_merge = pd.merge(
    df_mlb,
    df_odds,
    how='left',
    left_on=['date', 'home_abbr', 'away_abbr', 'home_score', 'away_score'],
    right_on=['date', 'home_team_abbr', 'away_team_abbr', 'home_score', 'away_score'],
    suffixes=('', '_odds'),
    indicator=True
)

missing_nym = debug_merge[
    (debug_merge['_merge'] == 'left_only') &
    ((debug_merge['home_abbr'] == 'NYM') | (debug_merge['away_abbr'] == 'NYM'))
]

print(f"Found {len(missing_nym)} missing Mets games.")
print("Dates and Scores in MLB API (that failed to match Odds):")
cols_to_show = ['date', 'home_abbr', 'away_abbr', 'home_score', 'away_score']
print(missing_nym[cols_to_show].sort_values('date'))

In [ ]:
df_2023_odds['date'].value_counts().sort_index()

In [ ]:
df_final_clean['away_abbr'][df_final_clean['away_abbr'] == 'NYM'].value_counts()

In [ ]:
df_final_clean['home_abbr'][df_final_clean['home_abbr'] == 'NYM'].value_counts()

In [ ]:



combined = pd.merge(
    df_mlb,
    df_odds,
    on=['date', 'home_abbr', 'away_abbr', 'home_score', 'away_score'],
    how='left'
)

print(f"MLB games: {len(df_mlb)}")
print(f"Odds data: {len(df_odds)}")
print(f"Matched: {len(combined)}")
combined[combined['date'] == '2023-03-30']

In [ ]:
df_merged = combined
save_path = f"{project_dir}/mlb_data_master.parquet"

df_merged.to_parquet(save_path)
print(f"Saved: {save_path}")

In [ ]:
import json
with open(loader.filepath, 'r') as f:
    raw_data = json.load(f)

first_date = list(raw_data.keys())[0]
first_game = raw_data[first_date][0]

print(json.dumps(first_game, indent=2))

In [ ]:
import json
import pandas as pd
from collections import defaultdict

def audit_sportsbook_coverage(filepath: str):
    print(f"Auditing sportsbook coverage in {filepath}...")

    with open(filepath, 'r') as f:
        raw_data = json.load(f)

    if not isinstance(raw_data, dict):
        print("Data is not a dictionary (unexpected format).")
        return

    stats = defaultdict(lambda: defaultdict(int))
    total_games = defaultdict(int)

    for date_key, games in raw_data.items():
        year = date_key[:4]

        for game in games:
            total_games[year] += 1

            odds = game.get('odds', {}).get('moneyline', [])
            for book_entry in odds:
                book_name = book_entry.get('sportsbook')
                if book_name:
                    stats[year][book_name] += 1

    df_audit = pd.DataFrame(stats).fillna(0).astype(int).T

    df_audit['Total_Games_Available'] = df_audit.index.map(total_games)

    df_audit = df_audit.sort_index()

    return df_audit

audit_df = audit_sportsbook_coverage("/content/drive/MyDrive/mlb-project-data/mlb_odds_dataset.json")

print("Rows = Years, Columns = Sportsbooks (Count of games covered)")
audit_df

In [ ]:
import json
import pandas as pd
from collections import defaultdict

def audit_game_types(filepath, target_year='2023'):
    print(f"Odds by Game Type for {target_year}...")

    with open(filepath, 'r') as f:
        raw_data = json.load(f)

    stats = defaultdict(lambda: {'total': 0, 'with_odds': 0})

    for date_key, games in raw_data.items():

        if target_year not in date_key:
            continue

        for game in games:

            g_type = game.get('gameView', {}).get('gameType', 'Unknown')

            moneyline = game.get('odds', {}).get('moneyline', [])
            has_odds = 1 if moneyline else 0

            stats[g_type]['total'] += 1
            stats[g_type]['with_odds'] += has_odds

    df = pd.DataFrame(stats).T
    df['missing_odds'] = df['total'] - df['with_odds']
    df['coverage_pct'] = (df['with_odds'] / df['total'] * 100).round(1)

    return df.sort_values('total', ascending=False)

df_diagnosis = audit_game_types("/content/drive/MyDrive/mlb-project-data/mlb_odds_dataset.json", target_year="2023")
print(df_diagnosis)